<a href="https://colab.research.google.com/github/ben854719/Biometric-Aware-Fraud-Risk-Dashboard-with-Agentic-AI/blob/main/Agentic_AI_Avatar_FRAUD_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph

In [ ]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [ ]:
!cd mcp-server-demo
!ls

In [ ]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
from langgraph.graph import StateGraph, END
from IPython.display import display, HTML
from typing import TypedDict
import os
import random
from mcp.server.fastmcp import FastMCP
from google.colab import userdata

# Import API Key.
api_key = userdata.get("Ben856")  # Or paste your key directly
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")
os.environ["GOOGLE_API_KEY"] = api_key

# Ini Gemini Model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)
prompt = ChatPromptTemplate.from_template("Respond confidently to: {input}")
chain = prompt | llm

#  Dynamic HeyGen Video Selector.
def select_heygen_url(response_text: str) -> str:
    if "onboarding" in response_text.lower():
        return "https://app.heygen.com/videos/your_onboarding_video_id"
    elif "risk" in response_text.lower() or "concern" in response_text.lower():
        return "https://app.heygen.com/videos/your_fallback_video_id"
    else:
        return "https://app.heygen.com/videos/cd56d01dad544759ba6e872b093a32d8"  # Default

# 🎬 Avatar Node (Colab-safe link preview)
def avatar_node(input_text):
    print("Avatar says:", input_text)
    heygen_url = select_heygen_url(input_text)
    display(HTML(f'<a href="{heygen_url}" target="_blank">▶️ Click to view avatar response</a>'))
    return {"avatar_output": f"Avatar says: {input_text}"}

# 🛠️ Optional Tool Wrapper
avatar_tool = Tool.from_function(
    func=avatar_node,
    name="AvatarSpeaker",
    description="Trigger avatar speech"
)

# 🧩 Workflow Nodes
def gemini_node(state: "WorkflowState"):
    response = chain.invoke({"input": state["input"]})
    state["response"] = response.content
    return state

def emotion_node(state: "WorkflowState"):
    text = state["response"]
    if "confident" in text or "welcome" in text:
        state["emotion"] = "positive"
    elif "concern" in text or "risk" in text:
        state["emotion"] = "negative"
    else:
        state["emotion"] = "neutral"
    return state

def fallback_node(state: "WorkflowState"):
    if state["emotion"] == "negative":
        state["response"] = "Fallback message: Let's review the risk indicators together."
    return state

def onboarding_node(state: "WorkflowState"):
    if "onboarding" in state["response"].lower():
        state["response"] = "Welcome aboard! Here's how we ensure compliance and clarity."
    return state

def compliance_score_node(state: "WorkflowState"):
    score = random.randint(70, 100)
    state["response"] += f" (Compliance Score: {score})"
    return state

def avatar_speak_node(state: "WorkflowState"):
    avatar_result = avatar_node(state["response"])
    state["avatar_output"] = avatar_result["avatar_output"]
    return state

# 🧠 State Schema
class WorkflowState(TypedDict):
    input: str
    response: str
    emotion: str
    avatar_output: str

# 🔄 LangGraph Flow
graph = StateGraph(WorkflowState)
graph.add_node("gemini", gemini_node)
graph.add_node("emotion", emotion_node)
graph.add_node("fallback", fallback_node)
graph.add_node("onboarding", onboarding_node)
graph.add_node("compliance_score", compliance_score_node)
graph.add_node("avatar_speak", avatar_speak_node)

graph.set_entry_point("gemini")
graph.add_edge("gemini", "emotion")
graph.add_edge("emotion", "fallback")
graph.add_edge("fallback", "onboarding")
graph.add_edge("onboarding", "compliance_score")
graph.add_edge("compliance_score", "avatar_speak")
graph.add_edge("avatar_speak", END)

flow = graph.compile()

# 🚀 Run the Flow
initial_state = {"input": "Generate a confident greeting for onboarding and fraud detection"}
flow.invoke(initial_state)